<a href="https://colab.research.google.com/github/natu-n/natu-n/blob/main/5_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python -V
# Python 3.7.13 (5/8)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
from datetime import datetime
from datetime import timedelta

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
ss_url = "https://docs.google.com/spreadsheets/d/1-Qdl4HN7XDUEczbAf-6OBqQ5hxwmehP7BoEr55X1Wvw/edit#gid=156673541"

workbook = gc.open_by_url(ss_url)
worksheet = workbook.get_worksheet(0)


In [ ]:

## The xy format accepts [{ x: category/date, y: [min, q1, median, q3, max] }].

df3 = pd.DataFrame(worksheet.get_all_values())
df3.columns = list(df3.loc[0, :])
df3.drop(0, inplace=True)
df3['date'] = pd.to_datetime(df3['date'])
df3.set_index('date', inplace=True)

df3['systolic'] = df3['systolic'].astype('int')
df3['diastolic'] = df3['diastolic'].astype('int')

start = datetime.strptime('2017-10-01', '%Y-%m-%d')
end   = datetime.strptime('2017-10-18', '%Y-%m-%d')
days_num = (end - start).days +1

datelist = [start + timedelta(days=x) for x in range(days_num)]

df2 = pd.DataFrame({'date': datelist ,'systolic': np.NaN,'diastolic': np.NaN})
df2['date'] = pd.to_datetime(df2['date'])
df2.set_index('date', inplace=True)

df = df2.append(df3, ignore_index=False)


# The xy format accepts [{ x: category/date, y: [min, q1, median, q3, max] }].

df_systolic = df.drop(columns='diastolic')
df_diastolic = df.drop(columns='systolic')
df_d = []


In [ ]:
## resample('M')は常に月末
## Min ##
df_w = df_diastolic.resample('M').min()
df_w.reset_index(inplace = True)
df_d.append(df_w.rename(columns={'diastolic': 'Min'}))

## Q! ##
df_w = df_diastolic.resample('M').quantile([0.25])
df_w.reset_index(inplace = True)
df_w = df_w.drop(columns='level_1')
df_w['diastolic'] = df_w['diastolic'].astype('float')
df_d.append(df_w.rename(columns={'diastolic': 'Q1'}))

## median ##
df_w = df_diastolic.resample('M').mean()
df_w.reset_index(inplace = True)
df_d.append(df_w.rename(columns={'diastolic': 'Mean'}))

## Q3 ##
df_w = df_diastolic.resample('M').quantile([0.75])
df_w.reset_index(inplace = True)
df_w = df_w.drop(columns='level_1')
df_w['diastolic'] = df_w['diastolic'].astype('float')
df_d.append(df_w.rename(columns={'diastolic': 'Q3'}))

## Max ##
df_w = df_diastolic.resample('M').max()
df_w.reset_index(inplace = True)
df_d.append(df_w.rename(columns={'diastolic': 'Max'}))


In [ ]:
## df_sheet_diastolicのガワの作成
df_sheet_diastolic = df_d[0]
df_sheet_diastolic = df_sheet_diastolic.drop(columns='Min')

## 纏める
for i in range(len(df_d)):
  df_sheet_diastolic = pd.merge(df_sheet_diastolic, df_d[i])


In [ ]:
## iqrを求める ##
def comp_iqr(df_sheet_diastolic):
  result = df_sheet_diastolic['Q3'] - df_sheet_diastolic['Q1']
  return result
## Low
def comp_low(df_sheet_diastolic):
  result = df_sheet_diastolic['Q1'] - (1.5 * df_sheet_diastolic['Iqr'])
  return result
## High
def comp_high(df_sheet_diastolic):
  result = df_sheet_diastolic['Q3'] + (1.5 * df_sheet_diastolic['Iqr'])
  return result

df_sheet_diastolic['Iqr'] = df_sheet_diastolic.apply(comp_iqr, axis=1)

df_sheet_diastolic['low'] = df_sheet_diastolic.apply(comp_low, axis=1)
df_sheet_diastolic['high'] = df_sheet_diastolic.apply(comp_high, axis=1)

df_sheet_diastolic.head(5)

+ [X] TimestampをMMM-YYYYに変換する
1. [x] resample('M')でQ1とQ3を求める
1. [x] Iqrを求める
+ [X] 外れ値
+ [ ] resample('M')のキャンセル
```
df_systolic.resample("M").min().index
DatetimeIndex(['2017-10-31', '2017-11-30', '2017-12-31', '2018-01-31',
df_systolic.resample("M",closed= "left", label="left").min().index
DatetimeIndex(['2017-09-30', '2017-10-31', '2017-11-30', '2017-12-31',ss
```
+ 2017-10-19
+ [x]2017-10-01 - 2017-10-18までのnp.NaNデータを混ぜ込むことは出来た
+ [x]必要な項目の取得方法はわかった


In [ ]:
stDate = '2018-11-01'
edDate = '2018-11-30'
df_systolic.loc[ stDate : edDate ]  # 直観的でよい


In [ ]:
#df_sheet_diastolic.index
cnt = df_sheet_diastolic['Min'].count()
df_sheet_diastolic.head(5)

|index|date|Min|Q1|Mean|Q3|Max|Iqr|low|high|
|---|---|---|---|---|---|---|---|---|---|
|0|2017-10-31 00:00:00|79\.0|90\.0|91\.6923076923077|94\.0|100\.0|4\.0|84\.0|100\.0|
|1|2017-11-30 00:00:00|79\.0|88\.0|92\.5|94\.75|107\.0|6\.75|77\.875|104\.875|
|2|2017-12-31 00:00:00|83\.0|89\.0|91\.7741935483871|93\.5|103\.0|4\.5|82\.25|100\.25|
|3|2018-01-31 00:00:00|84\.0|92\.0|95\.54838709677419|100\.0|110\.0|8\.0|80\.0|112\.0|
|4|2018-02-28 00:00:00|72\.0|97\.0|101\.10714285714286|104\.25|119\.0|7\.25|86\.125|115\.125|

## とりあえず下の外れ値、上の外れ値のDFを作る
## のと同時に外れ値を取り除く

In [ ]:
stDate = '2018-11-01'
edDate = '2018-11-30'
df_diastolic.loc[ stDate : edDate ]  # 直観的でよい


In [ ]:
#
# resampleした日付をもとに対象範囲を求める
#
stDate = df_sheet_diastolic['date'][4].strftime('%Y-%m-01') # これで月初日とは…
edDate = df_sheet_diastolic['date'][4].strftime('%Y-%m-%d')
high = df_sheet_diastolic['high'][4]
low = df_sheet_diastolic['low'][4]
print(high)
print(low)
df_diastolic['low'] = df_diastolic.loc[ stDate : edDate ]['diastolic'].apply(lambda x: low > x)
df_diastolic['high'] = df_diastolic.loc[ stDate : edDate ]['diastolic'].apply(lambda x: high < x)


In [ ]:
df_diastolic.loc[ stDate : edDate ] 